In [1]:
import pandas as pd

In [2]:
prefix = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/'
url = f'{prefix}/yellow_tripdata_2021-01.csv.gz'
url

'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow//yellow_tripdata_2021-01.csv.gz'

In [5]:
df = pd.read_csv(url)

/tmp/ipykernel_5155/4176769558.py:1: DtypeWarning: Columns (0: store_and_fwd_flag) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [9]:
dtype = {
    "VendorID": "Int64",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "RatecodeID": "Int64",
    "store_and_fwd_flag": "string",
    "PULocationID": "Int64",
    "DOLocationID": "Int64",
    "payment_type": "Int64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "congestion_surcharge": "float64"
}

parse_dates = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
]

df = pd.read_csv(
    url,
    dtype=dtype,
    parse_dates=parse_dates
)

In [11]:
df['tpep_pickup_datetime']

0         2021-01-01 00:30:10
1         2021-01-01 00:51:20
2         2021-01-01 00:43:30
3         2021-01-01 00:15:48
4         2021-01-01 00:31:49
                  ...        
1369760   2021-01-25 08:32:04
1369761   2021-01-25 08:34:00
1369762   2021-01-25 08:37:00
1369763   2021-01-25 08:28:00
1369764   2021-01-25 08:38:00
Name: tpep_pickup_datetime, Length: 1369765, dtype: datetime64[us]